In [1]:
######################################     Modules     #######################################
# MODULES
import sys
from time import process_time

import verbose_module
sys.path.append('../../.')
import pIRPgym


########################     Instance generator and Environment     #########################
### pIRP model's parameters
# Stochasticity
stochastic_params = ['d','q']
look_ahead = ['d','q']


# Historical data
historical_data = ['*']


# Other parameters
backorders = 'backorders'

# Random Instance
q_params = {'dist': 'c_uniform', 'r_f_params': [6,20]}          # Offer
p_params = {'dist': 'd_uniform', 'r_f_params': [20,61]}

d_params = {'dist': 'log-normal', 'r_f_params': [3,1]}          # Demand

h_params = {'dist': 'd_uniform', 'r_f_params': [20,61]}         # Holding costs

disc = ("strong","conc")


### Environment 
# Creating environment object
routing = True
inventory = True
perishability = 'ages'
env = pIRPgym.steroid_IRP(routing, inventory, perishability)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-01


# M = 12, K = 12

In [2]:
#%%
################################## Policy Evaluation ##################################
env_config = {'M':12,'K':12,'T':8,'F':14,'Q':1500,
              'S':6,'LA_horizon':4,
             'd_max':2500,'hist_window':60,
             'back_o_cost':5000}

stoch_rd_seed = 15       # Random seeds
det_rd_seed = 35

# Creating instance generator object
inst_gen = pIRPgym.instance_generator(look_ahead, stochastic_params,
                              historical_data, backorders, env_config = env_config)

inst_gen.generate_basic_random_instance(det_rd_seed,stoch_rd_seed,q_params=q_params,
            p_params=p_params,d_params=d_params,h_params=h_params,discount=disc)

# Reseting the environment
state = env.reset(inst_gen,return_state=True)

''' Parameters '''
verbose = True
start = process_time()
string = str()

if verbose: string = verbose_module.CG_initialization.print_head()

# Reseting the environment
state = env.reset(inst_gen,return_state=True)

alphas = [0.1,0.2,0.4,0.6]

done = False
while not done:
    ''' Purchase'''
    [purchase,demand_compliance], la_dec = pIRPgym.Inventory.Stochastic_Rolling_Horizon(state,env,inst_gen)

    if verbose: string = verbose_module.CG_initialization.print_step(env.t,purchase)
    
    ''' Routing '''
    CG_routes,CG_obj,CG_info,CG_time,CG_cols = pIRPgym.Routing.ColumnGeneration(purchase,inst_gen,env.t,time_limit=1800,
                                                                                verbose=False,return_num_cols=True)       # Column Generation algorithm    
    if verbose: string = verbose_module.CG_initialization.print_update(string,CG_time,CG_cols[1],len(CG_routes),CG_obj)

    for alpha in alphas:
        end = False
        if alpha == alphas[-1]:
            end = True
        CGinit_routes,CGinit_obj,CGinit_info,CGinit_time,CGinit_cols = pIRPgym.Routing.ColumnGeneration(purchase,inst_gen,env.t,time_limit=False,
                                                                                                        verbose=False,heuristic_initialization=True,
                                                                                                        return_num_cols=True,RCL_alpha=alpha)       # Column Generation algorithm                  
        if verbose: string = verbose_module.CG_initialization.print_update(string,CGinit_time,CGinit_cols,len(CGinit_routes),CGinit_obj,end=end)


    ''' Compound action'''        
    action = {'routing':CG_routes,'purchase':purchase,'demand_compliance':demand_compliance}

    state, reward, done, real_action, _,  = env.step(action,inst_gen)


print('Finished episode!!!')


*****************************************************************************  Evaluation of Heuristic Initialization on CG  ****************************************************************************
--------|	  	CG 	 	|	  CG w/ Init (alpha = 0.1) 	|	  CG w/ Init (alpha = 0.2) 	|	  CG w/ Init (alpha = 0.4) 	|	  CG w/ Init (alpha = 0.6) 	|
t    M	| t(s)	 cols	 #Veh 	 Obj 	| t(s)	RCLcols	 cols	 #Veh 	 Obj 	| t(s)	RCLcols	 cols	 #Veh 	 Obj 	| t(s)	RCLcols	 cols	 #Veh 	 Obj 	| t(s)	RCLcols	 cols	 #Veh 	 Obj 	|
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
0    10	| 6.78	  57 	   3 	6007.6 	| 13.58	   3 	  52 	   3 	5999.6 	| 13.42	   6 	  38 	   3 	5999.6 	| 12.84	   85 	  13 	   3 	5999.6 	| 12.42	   287 	  10 	   3 	6000.1 	|
1    8	| 0.64	  24 	   3 	5852.4 	| 5.75	   5 	  26 	   3 	5852.3 	| 5.70	   7 	  26 	   3 	5852.3 	| 6.72	 

# M = 15, K = 15

In [2]:
#%%
################################## Policy Evaluation ##################################
env_config = {'M':15,'K':15,'T':7,'F':14,'Q':1500,
              'S':6,'LA_horizon':4,
             'd_max':2500,'hist_window':60,
             'back_o_cost':5000}

stoch_rd_seed = 15       # Random seeds
det_rd_seed = 35

# Creating instance generator object
inst_gen = pIRPgym.instance_generator(look_ahead, stochastic_params,
                              historical_data, backorders, env_config = env_config)

inst_gen.generate_basic_random_instance(det_rd_seed,stoch_rd_seed,q_params=q_params,
            p_params=p_params,d_params=d_params,h_params=h_params,discount=disc)

# Reseting the environment
state = env.reset(inst_gen,return_state=True)

''' Parameters '''
verbose = True
start = process_time()
string = str()

if verbose: string = verbose_module.CG_initialization.print_head()

# Reseting the environment
state = env.reset(inst_gen,return_state=True)

alphas = [0.1,0.2,0.4,0.6]

done = False
while not done:
    ''' Purchase'''
    [purchase,demand_compliance], la_dec = pIRPgym.Inventory.Stochastic_Rolling_Horizon(state,env,inst_gen)

    if verbose: string = verbose_module.CG_initialization.print_step(env.t,purchase)
    
    ''' Routing '''
    CG_routes,CG_obj,CG_info,CG_time,CG_cols = pIRPgym.Routing.ColumnGeneration(purchase,inst_gen,env.t,time_limit=1800,
                                                                                verbose=False,return_num_cols=True)       # Column Generation algorithm    
    if verbose: string = verbose_module.CG_initialization.print_update(string,CG_time,CG_cols[1],len(CG_routes),CG_obj)

    for alpha in alphas:
        end = False
        if alpha == alphas[-1]:
            end = True
        CGinit_routes,CGinit_obj,CGinit_info,CGinit_time,CGinit_cols = pIRPgym.Routing.ColumnGeneration(purchase,inst_gen,env.t,time_limit=False,
                                                                                                        verbose=False,heuristic_initialization=True,
                                                                                                        return_num_cols=True,RCL_alpha=alpha)       # Column Generation algorithm                  
        if verbose: string = verbose_module.CG_initialization.print_update(string,CGinit_time,CGinit_cols,len(CGinit_routes),CGinit_obj,end=end)


    ''' Compound action'''        
    action = {'routing':CG_routes,'purchase':purchase,'demand_compliance':demand_compliance}

    state, reward, done, real_action, _,  = env.step(action,inst_gen)


print('Finished episode!!!')


*****************************************************************************  Evaluation of Heuristic Initialization on CG  ****************************************************************************
--------|	  	CG 	 	|	  CG w/ Init (alpha = 0.1) 	|	  CG w/ Init (alpha = 0.2) 	|	  CG w/ Init (alpha = 0.4) 	|	  CG w/ Init (alpha = 0.6) 	|
t    M	| t(s)	 cols	 #Veh 	 Obj 	| t(s)	RCLcols	 cols	 #Veh 	 Obj 	| t(s)	RCLcols	 cols	 #Veh 	 Obj 	| t(s)	RCLcols	 cols	 #Veh 	 Obj 	| t(s)	RCLcols	 cols	 #Veh 	 Obj 	|
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
0    11	| 1615	  176 	   3 	6073.8 	| 76.62	  3	  50	   3 	6073.8 	| 47.94	  12	  40	   3 	6073.8 	| 49.25	  48	  40	   3 	6073.8 	| 98.22	  341	  25	   3 	6073.8 	|
1    8	| 0.67	  33 	   3 	5963.6 	| 5.27	  5	  10	   3 	5963.6 	| 5.50	  13	  12	   3 	5963.6 	| 5.70	  14	  12	   3 	

# M = 18, K = 18

In [2]:
#%%
################################## Policy Evaluation ##################################
env_config = {'M':18,'K':18,'T':7,'F':18,'Q':1500,
              'S':6,'LA_horizon':4,
             'd_max':2500,'hist_window':60,
             'back_o_cost':5000}

stoch_rd_seed = 16       # Random seeds
det_rd_seed = 36

# Creating instance generator object
inst_gen = pIRPgym.instance_generator(look_ahead, stochastic_params,
                              historical_data, backorders, env_config = env_config)

inst_gen.generate_basic_random_instance(det_rd_seed,stoch_rd_seed,q_params=q_params,
            p_params=p_params,d_params=d_params,h_params=h_params,discount=disc)

# Reseting the environment
state = env.reset(inst_gen,return_state=True)

''' Parameters '''
verbose = True
start = process_time()
string = str()

if verbose: string = verbose_module.CG_initialization.print_head()

# Reseting the environment
state = env.reset(inst_gen,return_state=True)

alphas = [0.1,0.2,0.4,0.6]

done = False
while not done:
    ''' Purchase'''
    [purchase,demand_compliance], la_dec = pIRPgym.Inventory.Stochastic_Rolling_Horizon(state,env,inst_gen)

    if verbose: string = verbose_module.CG_initialization.print_step(env.t,purchase)
    
    ''' Routing '''
    CG_routes,CG_obj,CG_info,CG_time,CG_cols = pIRPgym.Routing.ColumnGeneration(purchase,inst_gen,env.t,time_limit=1800,
                                                                                verbose=False,return_num_cols=True)       # Column Generation algorithm    
    if verbose: string = verbose_module.CG_initialization.print_update(string,CG_time,CG_cols[1],len(CG_routes),CG_obj)

    for alpha in alphas:
        end = False
        if alpha == alphas[-1]:
            end = True
        CGinit_routes,CGinit_obj,CGinit_info,CGinit_time,CGinit_cols = pIRPgym.Routing.ColumnGeneration(purchase,inst_gen,env.t,time_limit=1800,
                                                                                                        verbose=False,heuristic_initialization=True,
                                                                                                        return_num_cols=True,RCL_alpha=alpha)       # Column Generation algorithm                  
        if verbose: string = verbose_module.CG_initialization.print_update(string,CGinit_time,CGinit_cols,len(CGinit_routes),CGinit_obj,end=end)


    ''' Compound action'''        
    action = {'routing':CG_routes,'purchase':purchase,'demand_compliance':demand_compliance}

    state, reward, done, real_action, _,  = env.step(action,inst_gen)


print('Finished episode!!!')


*****************************************************************************  Evaluation of Heuristic Initialization on CG  ****************************************************************************
--------|	  	CG 	 	|	  CG w/ Init (alpha = 0.1) 	|	  CG w/ Init (alpha = 0.2) 	|	  CG w/ Init (alpha = 0.4) 	|	  CG w/ Init (alpha = 0.6) 	|
t    M	| t(s)	 cols	 #Veh 	 Obj 	| t(s)	RCLcols	 cols	 #Veh 	 Obj 	| t(s)	RCLcols	 cols	 #Veh 	 Obj 	| t(s)	RCLcols	 cols	 #Veh 	 Obj 	| t(s)	RCLcols	 cols	 #Veh 	 Obj 	|
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
0    16	| 1870	  293 	   2 	4309.2 	| 441.4	  29	  107	   2 	4327.6 	| 843.9	  736	  112	   2 	4316.6 	| 901.8	  36241	  39	   2 	4309.2 	| 745.7	  49739	  66	   2 	4309.2 	|
1    17	| 1958	  313 	   2 	4361.9 	| 508.2	  62	  145	   2 	4236.6 	| 1959	  6352	  70	   2 	4203.6 	| 1842	

# M = 25, K = 25

In [ ]:
#%%
################################## Policy Evaluation ##################################
env_config = {'M':25,'K':25,'T':5,'F':25,'Q':1500,
              'S':6,'LA_horizon':4,
             'd_max':2500,'hist_window':60,
             'back_o_cost':5000}

stoch_rd_seed = 116       # Random seeds
det_rd_seed = 136

# Creating instance generator object
inst_gen = pIRPgym.instance_generator(look_ahead, stochastic_params,
                              historical_data, backorders, env_config = env_config)

inst_gen.generate_basic_random_instance(det_rd_seed,stoch_rd_seed,q_params=q_params,
            p_params=p_params,d_params=d_params,h_params=h_params,discount=disc)

# Reseting the environment
state = env.reset(inst_gen,return_state=True)

''' Parameters '''
verbose = True
start = process_time()
string = str()

if verbose: string = verbose_module.CG_initialization.print_head()

# Reseting the environment
state = env.reset(inst_gen,return_state=True)

alphas = [0.1,0.2,0.4,0.6]

done = False
while not done:
    ''' Purchase'''
    [purchase,demand_compliance], la_dec = pIRPgym.Inventory.Stochastic_Rolling_Horizon(state,env,inst_gen)

    if verbose: string = verbose_module.CG_initialization.print_step(env.t,purchase)
    
    ''' Routing '''
    CG_routes,CG_obj,CG_info,CG_time,CG_cols = pIRPgym.Routing.ColumnGeneration(purchase,inst_gen,env.t,time_limit=1800,
                                                                                verbose=False,return_num_cols=True)       # Column Generation algorithm    
    if verbose: string = verbose_module.CG_initialization.print_update(string,CG_time,CG_cols[1],len(CG_routes),CG_obj)

    for alpha in alphas:
        end = False
        if alpha == alphas[-1]:
            end = True
        CGinit_routes,CGinit_obj,CGinit_info,CGinit_time,CGinit_cols = pIRPgym.Routing.ColumnGeneration(purchase,inst_gen,env.t,time_limit=1800,
                                                                                                        verbose=False,heuristic_initialization=True,
                                                                                                        return_num_cols=True,RCL_alpha=alpha)       # Column Generation algorithm                  
        if verbose: string = verbose_module.CG_initialization.print_update(string,CGinit_time,CGinit_cols,len(CGinit_routes),CGinit_obj,end=end)


    ''' Compound action'''        
    action = {'routing':CG_routes,'purchase':purchase,'demand_compliance':demand_compliance}

    state, reward, done, real_action, _,  = env.step(action,inst_gen)


print('Finished episode!!!')
